In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [3]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [4]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [5]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [6]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.28134798
====> Test set loss: 1.2909, 62.0%
====> Epoch: 150 Average loss: 1.20173033
====> Test set loss: 1.2127, 69.0%
====> Epoch: 225 Average loss: 1.20760253
====> Test set loss: 1.2092, 69.0%
====> Epoch: 300 Average loss: 1.21259918
====> Test set loss: 1.2043, 69.0%
====> Epoch: 375 Average loss: 1.20176897
====> Test set loss: 1.2014, 70.0%
====> Epoch: 450 Average loss: 1.24033634
====> Test set loss: 1.2018, 69.5%
====> Epoch: 525 Average loss: 1.24107657
====> Test set loss: 1.1998, 70.5%
====> Epoch: 600 Average loss: 1.23011834
====> Test set loss: 1.1991, 70.5%
====> Epoch: 675 Average loss: 1.22299410
====> Test set loss: 1.1984, 70.5%
====> Epoch: 750 Average loss: 1.21179613
====> Test set loss: 1.1980, 70.5%
Training state:  False
Elapsed:  46.78293299674988
Complete set accuracy: 72.0%


### Runner

In [7]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(474, 500):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 450
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.29434011
====> Test set loss: 1.2049, 71.5%
====> Epoch: 150 Average loss: 1.22425010
====> Test set loss: 1.1300, 71.0%
====> Epoch: 225 Average loss: 1.22711407
====> Test set loss: 1.1418, 71.5%
====> Epoch: 300 Average loss: 1.20731689
====> Test set loss: 1.1459, 71.5%
====> Epoch: 375 Average loss: 1.25387398
====> Test set loss: 1.1428, 71.5%
====> Epoch: 450 Average loss: 1.23433218
====> Test set loss: 1.1436, 71.5%
====> Epoch: 525 Average loss: 1.24365929
====> Test set loss: 1.1444, 71.5%
====> Epoch: 600 Average loss: 1.22916873
====> Test set loss: 1.1449, 71.5%
====> Epoch: 675 Average loss: 1.23274640
====> Test set loss: 1.1452, 71.5%
====> Epoch: 750 Average loss: 1.25016250
====> Test set loss: 1.1451, 71.5%
Training state:  False
Complete set accuracy: 71.2%
Log accuracy: 69.3%
---- Done in  55.7545862197876  seconds

---- Running for model name:  B_add_mild_nlin
===

====> Epoch: 675 Average loss: 1.22517361
====> Test set loss: 1.1227, 75.0%
====> Epoch: 750 Average loss: 1.25347376
====> Test set loss: 1.1222, 75.0%
Training state:  False
Complete set accuracy: 73.5%
Log accuracy: 71.0%
---- Done in  55.47582817077637  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.29868566
====> Test set loss: 1.2259, 69.0%
====> Epoch: 150 Average loss: 1.16687155
====> Test set loss: 1.1566, 69.0%
====> Epoch: 225 Average loss: 1.17449649
====> Test set loss: 1.1565, 69.5%
====> Epoch: 300 Average loss: 1.19451594
====> Test set loss: 1.1574, 69.5%
====> Epoch: 375 Average loss: 1.21210462
====> Test set loss: 1.1556, 69.5%
====> Epoch: 450 Average loss: 1.14100704
====> Test set loss: 1.1551, 69.0%
====> Epoch: 525 Average loss: 1.12961251
====> Test set loss: 1.1546, 69.0%
====> Epoch: 600 Average loss: 1.11154794
====> Test set loss: 1.1543, 69.0%
====> Epoch: 675 Average loss: 1.18120827
====> Test set loss: 1.1537, 6

====> Epoch: 450 Average loss: 1.14467265
====> Test set loss: 1.1227, 70.5%
====> Epoch: 525 Average loss: 1.11633135
====> Test set loss: 1.1232, 71.0%
====> Epoch: 600 Average loss: 1.16466413
====> Test set loss: 1.1237, 71.0%
====> Epoch: 675 Average loss: 1.17480198
====> Test set loss: 1.1241, 71.0%
====> Epoch: 750 Average loss: 1.18446242
====> Test set loss: 1.1238, 71.0%
Training state:  False
Complete set accuracy: 74.4%
Log accuracy: 73.2%
---- Done in  54.7332398891449  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.22229158
====> Test set loss: 1.1255, 76.5%
====> Epoch: 150 Average loss: 1.14083302
====> Test set loss: 1.0673, 77.5%
====> Epoch: 225 Average loss: 1.13359447
====> Test set loss: 1.0772, 77.0%
====> Epoch: 300 Average loss: 1.08760875
====> Test set loss: 1.0752, 77.0%
====> Epoch: 375 Average loss: 1.13017335
====> Test set loss: 1.0747, 77.0%
====> Epoch: 450 Average loss: 1.16332905
====> Test set loss: 1.0

====> Epoch: 225 Average loss: 1.22499331
====> Test set loss: 1.1258, 70.0%
====> Epoch: 300 Average loss: 1.11562216
====> Test set loss: 1.1271, 70.5%
====> Epoch: 375 Average loss: 1.09831358
====> Test set loss: 1.1256, 70.5%
====> Epoch: 450 Average loss: 1.15256802
====> Test set loss: 1.1266, 70.5%
====> Epoch: 525 Average loss: 1.12649822
====> Test set loss: 1.1265, 70.5%
====> Epoch: 600 Average loss: 1.13823044
====> Test set loss: 1.1261, 70.5%
====> Epoch: 675 Average loss: 1.17034978
====> Test set loss: 1.1266, 70.5%
====> Epoch: 750 Average loss: 1.11761451
====> Test set loss: 1.1265, 70.5%
Training state:  False
Complete set accuracy: 76.3%
Log accuracy: 72.5%
---- Done in  58.19610810279846  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.29471613
====> Test set loss: 1.2762, 63.5%
====> Epoch: 150 Average loss: 1.22709435
====> Test set loss: 1.1650, 69.0%
====> Epoch: 225 Average loss: 1.18874407
====> Test set loss: 1.17

====> Epoch: 75 Average loss: 1.30759189
====> Test set loss: 1.2895, 66.0%
====> Epoch: 150 Average loss: 1.17809259
====> Test set loss: 1.2281, 64.5%
====> Epoch: 225 Average loss: 1.19540823
====> Test set loss: 1.2337, 64.0%
====> Epoch: 300 Average loss: 1.20575749
====> Test set loss: 1.2340, 63.5%
====> Epoch: 375 Average loss: 1.16594755
====> Test set loss: 1.2377, 63.5%
====> Epoch: 450 Average loss: 1.21192615
====> Test set loss: 1.2380, 63.5%
====> Epoch: 525 Average loss: 1.19089290
====> Test set loss: 1.2385, 64.0%
====> Epoch: 600 Average loss: 1.15670783
====> Test set loss: 1.2387, 64.0%
====> Epoch: 675 Average loss: 1.17283824
====> Test set loss: 1.2387, 64.0%
====> Epoch: 750 Average loss: 1.18328835
====> Test set loss: 1.2390, 64.0%
Training state:  False
Complete set accuracy: 70.8%
Log accuracy: 70.5%
---- Done in  56.34013390541077  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.25711236
====> Test set loss: 1.1890, 7

...

====> Epoch: 150 Average loss: 1.21766266
====> Test set loss: 1.1714, 69.0%
====> Epoch: 225 Average loss: 1.21602381
====> Test set loss: 1.1692, 67.5%
====> Epoch: 300 Average loss: 1.22674333
====> Test set loss: 1.1645, 68.0%
====> Epoch: 375 Average loss: 1.16175513
====> Test set loss: 1.1607, 68.0%
====> Epoch: 450 Average loss: 1.24938172
====> Test set loss: 1.1604, 68.5%
====> Epoch: 525 Average loss: 1.22496897
====> Test set loss: 1.1607, 68.5%
====> Epoch: 600 Average loss: 1.21127118
====> Test set loss: 1.1610, 68.5%
====> Epoch: 675 Average loss: 1.18719782
====> Test set loss: 1.1609, 68.5%
====> Epoch: 750 Average loss: 1.19909656
====> Test set loss: 1.1608, 68.5%
Training state:  False
Complete set accuracy: 71.1%
Log accuracy: 69.0%
---- Done in  68.53290891647339  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.24601781
====> Test set loss: 1.2127, 70.0%
====> Epoch: 150 Average loss: 1.15510507
====> Test set loss: 1.1893, 

====> Epoch: 75 Average loss: 1.25308698
====> Test set loss: 1.1667, 77.0%
====> Epoch: 150 Average loss: 1.18966731
====> Test set loss: 1.1207, 76.0%
====> Epoch: 225 Average loss: 1.16445344
====> Test set loss: 1.1187, 77.5%
====> Epoch: 300 Average loss: 1.16568737
====> Test set loss: 1.1146, 77.0%
====> Epoch: 375 Average loss: 1.21688927
====> Test set loss: 1.1124, 77.0%
====> Epoch: 450 Average loss: 1.19155015
====> Test set loss: 1.1124, 77.0%
====> Epoch: 525 Average loss: 1.17384255
====> Test set loss: 1.1125, 77.0%
====> Epoch: 600 Average loss: 1.15360406
====> Test set loss: 1.1122, 77.0%
====> Epoch: 675 Average loss: 1.20570862
====> Test set loss: 1.1122, 77.0%
====> Epoch: 750 Average loss: 1.16981763
====> Test set loss: 1.1121, 77.0%
Training state:  False
Complete set accuracy: 76.5%
Log accuracy: 74.1%
---- Done in  66.64724397659302  seconds

---- Running for model name:  F_mod_nadd_lin
====> Epoch: 75 Average loss: 1.30261168
====> Test set loss: 1.3041, 63

====> Epoch: 750 Average loss: 1.22805702
====> Test set loss: 1.1302, 70.0%
Training state:  False
Complete set accuracy: 74.7%
Log accuracy: 71.3%
---- Done in  61.561574935913086  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.32006571
====> Test set loss: 1.3445, 59.0%
====> Epoch: 150 Average loss: 1.25787524
====> Test set loss: 1.2971, 66.0%
====> Epoch: 225 Average loss: 1.24823348
====> Test set loss: 1.2969, 65.5%
====> Epoch: 300 Average loss: 1.20209290
====> Test set loss: 1.2960, 66.5%
====> Epoch: 375 Average loss: 1.26380769
====> Test set loss: 1.2968, 66.0%
====> Epoch: 450 Average loss: 1.23342116
====> Test set loss: 1.2961, 66.5%
====> Epoch: 525 Average loss: 1.27567301
====> Test set loss: 1.2952, 66.5%
====> Epoch: 600 Average loss: 1.25140079
====> Test set loss: 1.2951, 66.5%
====> Epoch: 675 Average loss: 1.24943456
====> Test set loss: 1.2952, 66.5%
====> Epoch: 750 Average loss: 1.21167378
====> Test set loss: 1.2

====> Epoch: 525 Average loss: 1.16906469
====> Test set loss: 1.2167, 66.5%
====> Epoch: 600 Average loss: 1.15460906
====> Test set loss: 1.2170, 66.5%
====> Epoch: 675 Average loss: 1.17388855
====> Test set loss: 1.2171, 66.5%
====> Epoch: 750 Average loss: 1.16374245
====> Test set loss: 1.2172, 66.5%
Training state:  False
Complete set accuracy: 74.6%
Log accuracy: 71.3%
---- Done in  79.3117949962616  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.25459805
====> Test set loss: 1.2334, 72.5%
====> Epoch: 150 Average loss: 1.21855877
====> Test set loss: 1.1988, 70.0%
====> Epoch: 225 Average loss: 1.22998725
====> Test set loss: 1.1962, 69.5%


KeyboardInterrupt: 